# Imports

In [6]:
import sys
import pickle
from PyQt5.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QWidget, QLabel, QPushButton, QTextEdit, QComboBox
from PyQt5.QtGui import QPainter, QPen, QImage, QColor, QPixmap
from PyQt5.QtCore import Qt, QRect
import numpy as np
import cv2
from keras.models import load_model

# Creating prediction output map

In [7]:
ascii_map = {} 
# Mapping over the emnist-balanced-mapping.txt contianing the assci vlaues of every output of the ann
with open("emnist-balanced-mapping.txt", "r") as file: 
        lines = file.readlines() 
        for line in lines: 
                index, ascii_value = map(int, line.split()) 
                character = chr(ascii_value) 
                ascii_map[index] = character 
        

def letter(prediction): 
        return ascii_map[prediction]


# Creating Canvas Class (For Painting)

In [8]:
class Canvas(QWidget):
    def __init__(self):
        super().__init__()
        self.setFixedSize(400, 400)
        self.image = QImage(self.size(), QImage.Format_RGB32)
        self.image.fill(Qt.white)
        self.last_point = None
        self.pen_color = QColor(Qt.black)
        self.pen_width = 20

    def paintEvent(self, event):
        painter = QPainter(self)
        painter.drawImage(QRect(0, 0, self.width(), self.height()), self.image, self.image.rect())

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.last_point = event.pos()

    def mouseMoveEvent(self, event):
        if event.buttons() and Qt.LeftButton and self.last_point:
            painter = QPainter(self.image)
            painter.setPen(QPen(self.pen_color, self.pen_width, Qt.SolidLine, Qt.RoundCap, Qt.RoundJoin))
            painter.drawLine(self.last_point, event.pos())
            self.last_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        if event.button == Qt.LeftButton:
            self.last_point = None

    def clearCanvas(self):
        self.image.fill(Qt.white)
        self.update() 
    
    def predictCharacter(self, predict_image):
        # Convert QImage to numpy array
        img = self.image.convertToFormat(QImage.Format_Grayscale8)
        ptr = img.constBits()
        ptr.setsize(img.byteCount())
        arr = np.array(ptr).reshape(img.height(), img.width()) 

        # Threshold the image to binary
        _, binary = cv2.threshold(arr, 128, 255, cv2.THRESH_BINARY_INV)

        # Find contours of the characters
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Sort contours based on their x-coordinate
        contours = sorted(contours, key=lambda x: cv2.boundingRect(x)[0])

        predictions = []
        imgs=[]
        for contour in contours:
            # Get bounding box coordinates for each character
            x, y, w, h = cv2.boundingRect(contour)
            
            # Extract individual character using slicing and resize it to 28x28 pixels 
            char_image = arr[y:y+h, x:x+w]
            
            # Apply properties to the segment
            img_resized = cv2.resize(char_image, (28, 28))
            img_resized = cv2.bitwise_not(img_resized)
            img_resized = cv2.copyMakeBorder(img_resized, 5, 5, 5, 5, cv2.BORDER_CONSTANT, value=(0,0,0))
            img_resized = cv2.resize(img_resized, (28, 28))
            img_resized = cv2.flip(img_resized, 0)
            img_resized = np.rot90(img_resized, -1)
            imgs.append(img_resized)
            img_resized = img_resized / 255.0
            
            # Flatten image
            img_flattened = img_resized.reshape(1, 784)
            
            label = predict_image(img_flattened)
            
            # Append each character's image into a list 
            predictions.append(label)
        
        return predictions

# The GUI class

In [9]:
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("OCR GUI")
        self.setGeometry(100, 100, 600, 600)
        
        self.canvas = Canvas()
        self.label = QLabel("Predicted Text:")
        self.predicted_text = QTextEdit()
        self.predicted_text.setReadOnly(True)
        self.predicted_text.setFixedSize(200, 100)
        self.predicted_text.setAlignment(Qt.AlignTop)
        
        clear_button = QPushButton("Clear Canvas")
        clear_button.clicked.connect(self.canvas.clearCanvas)
        
        self.model_combobox = QComboBox()
        self.model_combobox.addItem("ocr-full.h5")
        self.model_combobox.addItem("rf_model.pkl")
        
        layout = QVBoxLayout()
        layout.addWidget(self.canvas)
        layout.addWidget(self.label)
        layout.addWidget(self.predicted_text)
        layout.addWidget(clear_button)
        layout.addWidget(self.model_combobox)
        
        central_widget = QWidget()
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)
        
        self.model_combobox.currentIndexChanged.connect(self.load_model)
        self.current_model = None
        self.load_model()  # Load default model
        
    def load_model(self):
        model_filename = self.model_combobox.currentText()
        if model_filename.endswith(".h5"):
            self.current_model = load_model(model_filename)
        elif model_filename.endswith(".pkl"):
            with open(model_filename, "rb") as f:
                self.current_model = pickle.load(f)
        else:
            print("Unsupported model format:", model_filename)
        
    def predict_image(self, img):
        prediction = self.current_model.predict(img)
        
        model_filename = self.model_combobox.currentText()
        if model_filename.endswith(".h5"): 
            predicted_index = np.argmax(prediction)
        else:
            predicted_index=prediction[0]
        return letter(predicted_index)
    
    def update_predicted_text(self, predictions):
        current_text = self.predicted_text.toPlainText()
        new_text = ''.join(predictions)
        self.predicted_text.setPlainText(current_text+ " " + new_text)

    def keyPressEvent(self, event):
        if event.key() == Qt.Key_Return or event.key() == Qt.Key_Enter:
            predictions = self.canvas.predictCharacter(self.predict_image)
            self.update_predicted_text(predictions)


# Main funciton

In [10]:
def main():
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())

if __name__ == "__main__":
    main()


1/1 [==============================] - 0s 16ms/step
